In [18]:
import pandas as pd
import numpy as np
from functools import reduce

## Demographics
contains patient demographic and administrative information, a single record for each event
report.

In [2]:
# import data
demo = pd.read_csv("../data/demo2023q2.csv", dtype={'to_mfr': str})

In [47]:
demo.columns

Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'i_f_code_num',
       'event_dt', 'event_dt_num', 'mfr_dt', 'mfr_dt_num', 'init_fda_dt',
       'init_fda_dt_num', 'fda_dt', 'fda_dt_num', 'rept_cod', 'rept_cod_num',
       'auth_num', 'mfr_num', 'mfr_sndr', 'lit_ref', 'age', 'age_cod',
       'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod', 'rept_dt', 'rept_dt_num',
       'to_mfr', 'occp_cod', 'reporter_country', 'occr_country',
       'occp_cod_num'],
      dtype='object')

In [9]:
# decide which columns to drop based on data dictionary: https://pharmahub.org/app/site/resources/2018/01/00739/FDA-FAERS-Data-Dictionary.pdf
to_drop = ["i_f_code", "i_f_code_num", "rept_cod", "rept_cod_num", "auth_num", "mfr_num", "e_sub", "rept_dt_num", "to_mfr" ]
demo.drop(to_drop, axis=1, inplace=True)

In [3]:
demo.head()

,primaryid,caseid,caseversion,i_f_code,i_f_code_num,event_dt,event_dt_num,mfr_dt,mfr_dt_num,init_fda_dt,...,e_sub,wt,wt_cod,rept_dt,rept_dt_num,to_mfr,occp_cod,reporter_country,occr_country,occp_cod_num
0,37318572,3731857,2,F,F Follow-up,19991101.0,01nov1999,20230515.0,15may2023,20011113,...,Y,NaN,NaN,20230518,18may2023,NaN,HP,US,US,NaN
1,40815622,4081562,2,F,F Follow-up,20011128.0,28nov2001,20230328.0,28mar2023,20040206,...,Y,52.0,KG,20230407,07apr2023,NaN,MD,DE,NaN,MD Physician
2,59214303,5921430,3,F,F Follow-up,20050122.0,22jan2005,20230420.0,20apr2023,20051111,...,Y,NaN,NaN,20230425,25apr2023,NaN,HP,US,NaN,NaN
3,59221653,5922165,3,F,F Follow-up,20050930.0,30sep2005,20230420.0,20apr2023,20051114,...,Y,103.0,KG,20230428,28apr2023,NaN,HP,CA,NaN,NaN
4,60665134,6066513,4,F,F Follow-up,NaN,NaN,20230531.0,31may2023,20060616,...,Y,NaN,NaN,20230602,02jun2023,NaN,HP,JP,JP,NaN


In [4]:
demo.shape

(418592, 33)

In [5]:
demo.value_counts("occr_country").head(n=10)

occr_country
US    244677
CA     28977
JP     14320
GB     12092
FR     11746
DE      7656
CN      5807
IT      5352
ES      3897
AU      3300
dtype: int64

In [6]:
demo.value_counts("mfr_sndr").head(n=10)

mfr_sndr
NOVARTIS                36480
PFIZER                  28864
SANOFI AVENTIS          23459
ABBVIE                  21329
ROCHE                   19707
AMGEN                   18685
JOHNSON AND JOHNSON     17993
BRISTOL MYERS SQUIBB    17660
FDA-CTU                 17197
UCB                     14300
dtype: int64

## Drug Information
contains drug/biologic information for as many medications as were reported for the event (1 or
more per event).

In [7]:
drug_info = pd.read_csv("../data/drug_info2023q2.csv")

In [31]:
drug_info.head()

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,37318572,3731857,1,PS,CYCLOSPORINE,CYCLOSPORINE,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,50574.0,NaN,NaN,NaN,NaN
1,37318572,3731857,2,SS,CYCLOSPORINE,CYCLOSPORINE,1,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,50574.0,NaN,NaN,NaN,NaN
2,37318572,3731857,3,SS,MYCOPHENOLATE MOFETIL,MYCOPHENOLATE MOFETIL,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37318572,3731857,4,SS,MYCOPHENOLATE MOFETIL,MYCOPHENOLATE MOFETIL,1,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37318572,3731857,5,SS,PREDNISONE,PREDNISONE,1,Unknown,UNK,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
to_drop_drug_info = ["nda_num"] #maybe - New Drug Application Number: This six digit number is assigned by FDA staff to each application for approval to market a new drug in the United States.

## Reaction Information
contains all "Medical Dictionary for Regulatory Activities" (MedDRA) terms coded for the adverse
event 

In [10]:
reac_info = pd.read_csv("../data/reac_info2023q2.csv")

In [11]:
reac_info.head()

,primaryid,caseid,pt,drug_rec_act
0,37318572,3731857,Apraxia,NaN
1,37318572,3731857,Confusional state,NaN
2,37318572,3731857,Depression,NaN
3,37318572,3731857,Glioblastoma multiforme,NaN
4,37318572,3731857,Memory impairment,NaN


In [12]:
to_drop_react_info = ["drug_rec_act"]

## Outcome Information
contains patient outcomes for the event

outc_cod key:

- DE - Death
- LT - Life-Threatening
- HO - Hospitalization - Initial or Prolonged
- DS - Disability
- CA - Congenital Anomaly
- RI - Required Intervention to PreventPermanent Impairment/Damage
- OT - Other Serious (Important Medical Event)

In [13]:
outcome = pd.read_csv("../data/outc2023q2.csv")

In [37]:
outcome.head()

,primaryid,caseid,outc_cod
0,37318572,3731857,DE
1,37318572,3731857,HO
2,40815622,4081562,HO
3,59214303,5921430,DE
4,59221653,5922165,DE


## Response Source 
(maybe not needed)

In [14]:
response_source = pd.read_csv("../data/rpsr2023q2.csv")
response_source.head()

,primaryid,caseid,rpsr_cod
0,221656031,22165603,CSM
1,221656041,22165604,CSM
2,221656051,22165605,CSM
3,221666201,22166620,CSM
4,221666211,22166621,CSM


## Therapy
contains drug therapy start dates and end dates for the reported drugs 

In [15]:
therapy = pd.read_csv("../data/ther2023q2.csv")
therapy.head()

,primaryid,caseid,dsg_drug_seq,start_dt,start_dt_num,end_dt,end_dt_num,dur,dur_cod
0,40815622,4081562,1,20011112.0,12nov2001,20011205.0,05dec2001,24.0,DAY
1,40815622,4081562,2,20011113.0,13nov2001,20011117.0,17nov2001,5.0,DAY
2,40815622,4081562,3,NaN,NaN,20011126.0,26nov2001,NaN,NaN
3,40815622,4081562,4,20011128.0,28nov2001,20011202.0,02dec2001,5.0,DAY
4,40815622,4081562,5,20011118.0,18nov2001,20011202.0,02dec2001,15.0,DAY


In [16]:
to_drop_therapy = ["start_dt", "start_dt_num", "end_dt", "end_dt_num"]

## MedDRA
contains all "Medical Dictionary for Regulatory Activities" (MedDRA) terms coded for the
indications for use (diagnoses) for the reported drugs

In [17]:
med_dra = pd.read_csv("../data/indi2023q2.csv")

In [42]:
med_dra.head()

,primaryid,caseid,indi_drug_seq,indi_pt
0,37318572,3731857,1,Renal transplant
1,37318572,3731857,2,Immunosuppressant drug therapy
2,37318572,3731857,3,Renal transplant
3,37318572,3731857,4,Immunosuppressant drug therapy
4,37318572,3731857,5,Renal transplant


## Merging

In [19]:
dataframes = [demo, drug_info, reac_info, outcome,response_source, therapy, med_dra]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['primaryid'],how='outer'), dataframes)

/var/folders/46/763y2yyd3jx4p7w2lscxy9l80000gp/T/ipykernel_2157/1846579592.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'caseid_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['primaryid'],how='outer'), dataframes)


In [52]:
df_merged.to_csv("../data/merged.csv", index=False)

In [53]:
df_merged.head()

,primaryid,caseid_x,caseversion,event_dt,event_dt_num,mfr_dt,mfr_dt_num,init_fda_dt,init_fda_dt_num,fda_dt,...,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,37318572,3731857,2,19991101.0,01nov1999,20230515.0,15may2023,20011113,13nov2001,20230518,...,NaN,U,NaN,NaN,NaN,50574.0,NaN,NaN,NaN,NaN
1,37318572,3731857,2,19991101.0,01nov1999,20230515.0,15may2023,20011113,13nov2001,20230518,...,NaN,U,NaN,NaN,NaN,50574.0,NaN,NaN,NaN,NaN
2,37318572,3731857,2,19991101.0,01nov1999,20230515.0,15may2023,20011113,13nov2001,20230518,...,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37318572,3731857,2,19991101.0,01nov1999,20230515.0,15may2023,20011113,13nov2001,20230518,...,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37318572,3731857,2,19991101.0,01nov1999,20230515.0,15may2023,20011113,13nov2001,20230518,...,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
